#**Setup**

Please note that if you accessed this notebook through the Colab button on GitHub then your data will be stored in an ephemeral session storage drive. Any important analysis should be saved before closing the notebook. In this case it might be more convenient to upload data directly into the session storage (Folder icon aka 'Files' > File upload icon aka 'Upload to session storage')

Sample input files are available at - https://github.com/Rayyan-Tariq-Khan/ProteoViz/tree/main/Input

You may also **optionally** download them directly into the ephemeral drive using the cell below-

In [ ]:
import urllib.request

# List of files with their URLs and local file names
files = [
    {
        "url": "https://raw.githubusercontent.com/Rayyan-Tariq-Khan/ProteoViz/main/Input/PXD037757.txt",
        "file_name": "PXD037757.txt"
    },
    {
        "url": "https://raw.githubusercontent.com/Rayyan-Tariq-Khan/ProteoViz/main/Input/Complexes.xlsx",
        "file_name": "Complexes.xlsx"
    }
]

# Download and save each file
for file in files:
    try:
        urllib.request.urlretrieve(file["url"], file["file_name"])
        print(f"File '{file['file_name']}' downloaded and saved successfully.")
    except Exception as e:
        print(f"An error occurred while downloading '{file['file_name']}': {e}")


#**Filtering**


This cell requires a full MaxQuant like ('Protein groups') .tsv output sheet with a .txt extension. It lets you filter out positive hits in these 3 columns - 'Reverse', 'Only identified by site', and 'Potential contaminant'.

Sample input files can be downloaded from the code cell above and they are also available at - https://github.com/Rayyan-Tariq-Khan/ProteoViz/tree/main/Input

**Important Note** - You may upload your own Input file, but kindly change the name of the 'input_file' variable to reflect this.

In [ ]:
import pandas as pd
import os

# Define the input and output file paths
input_file = '/content/PXD037757.txt'  # Input is now a TSV file with .txt extension
output_file = '/content/proteinGroups_filtered.txt'  # Output is a TSV file with .txt extension

# Check if the input file exists
if not os.path.exists(input_file):
    print(f"Input file '{input_file}' does not exist.")
else:
    # Load the TSV file into a pandas DataFrame
    df = pd.read_csv(input_file, sep='\t')
    print("TSV file loaded successfully.")

    # Print the initial number of rows
    initial_row_count = len(df)
    print(f"Initial number of rows: {initial_row_count}")

    # Check if the required columns exist
    required_columns = ['Only identified by site', 'Reverse', 'Potential contaminant']
    missing_columns = [col for col in required_columns if col not in df.columns]

    if not missing_columns:
        print("All required columns are present.")

        # Filter rows where 'Reverse' has a '+'
        df_filtered = df[df['Reverse'] != '+']
        print(f"Rows after filtering 'Reverse': {len(df_filtered)}")

        # Further remove rows where 'Only identified by site' has a '+'
        df_filtered = df_filtered[df_filtered['Only identified by site'] != '+']
        print(f"Rows after filtering 'Only identified by site': {len(df_filtered)}")

        # Further remove rows where 'Potential contaminant' has a '+'
        df_filtered = df_filtered[df_filtered['Potential contaminant'] != '+']
        print(f"Rows after filtering 'Potential contaminant': {len(df_filtered)}")

        # Save the filtered DataFrame to a new TSV file with .txt extension
        if not df_filtered.empty:
            df_filtered.to_csv(output_file, sep='\t', index=False)
            print(f"Filtered file has been saved as {output_file}")
        else:
            print("No data left after filtering. No file will be saved.")
    else:
        print(f"Missing columns: {', '.join(missing_columns)} not found in the file.")


This cell lets you keep 1 or 2 identifier columns from the original sheet. It also lets you select which abundance values to keep in the filtered sheet.

In [ ]:
import pandas as pd
import re
import os

# Define the input and output file paths
input_file = '/content/proteinGroups_filtered.txt'  # Input is now a TSV file with .txt extension
output_file = '/content/proteinGroups_filtered.txt'  # Output will be a TSV file with .txt extension

# Check if the input file exists
if not os.path.exists(input_file):
    print(f"Input file '{input_file}' does not exist.")
    exit()

# Load the TSV file into a pandas DataFrame
df = pd.read_csv(input_file, sep='\t')
print("TSV file loaded successfully.")

# Step 1: Ask the user for the number of identifier columns
num_identifiers = input("How many identifier columns do you have in your sheet? (Enter 1 or 2): ")

# Validate user input
if num_identifiers not in ['1', '2']:
    print("Invalid input. Please enter 1 or 2.")
    exit()

# Get the identifier columns from user input
identifier_columns = []
if num_identifiers == '1':
    identifier_column_name = input("Enter the header of the identifier column: ")
    if identifier_column_name not in df.columns:
        print(f"Column '{identifier_column_name}' not found in the sheet.")
        exit()
    identifier_columns = [identifier_column_name]
    id_prefix = '_1id'
else:
    identifier_column_name1 = input("Enter the header of the first identifier column: ")
    identifier_column_name2 = input("Enter the header of the second identifier column: ")
    if identifier_column_name1 not in df.columns or identifier_column_name2 not in df.columns:
        print("One or both of the specified identifier columns are not found in the sheet.")
        exit()
    identifier_columns = [identifier_column_name1, identifier_column_name2]
    id_prefix = '_2id'

# Replace empty cells in identifier columns with 'Missing ID'
df[identifier_columns] = df[identifier_columns].fillna('Missing ID')

# Step 2: Ask the user to choose the column series
print("Choose the column series you want to include:")
print("1. Intensity wtXX columns")
print("2. iBAQ wtXX columns")
print("3. Self-selected list of columns")
column_choice = input("Enter your choice (1, 2, or 3): ")

# Validate user input
if column_choice not in ['1', '2', '3']:
    print("Invalid input. Please enter 1, 2, or 3.")
    exit()

# Determine the series of columns to save based on user choice
if column_choice == '1':
    # Find the Intensity wtXX columns
    intensity_columns = [col for col in df.columns if re.match(r'^Intensity wt\d+$', col)]
    if intensity_columns:
        selected_columns = identifier_columns + intensity_columns
        column_prefix = '_Intensity'
    else:
        print("No 'Intensity wtXX' columns found.")
        exit()
elif column_choice == '2':
    # Find the iBAQ wtXX columns
    ibaq_columns = [col for col in df.columns if re.match(r'^iBAQ wt\d+$', col)]
    if ibaq_columns:
        selected_columns = identifier_columns + ibaq_columns
        column_prefix = '_IBAQ'
    else:
        print("No 'iBAQ wtXX' columns found.")
        exit()
elif column_choice == '3':
    # Ask user for self-selected columns
    first_col = input("Enter the header of the first column in the range: ")
    last_col = input("Enter the header of the last column in the range: ")

    if first_col in df.columns and last_col in df.columns:
        start_index = df.columns.get_loc(first_col)
        end_index = df.columns.get_loc(last_col)
        if start_index <= end_index:
            selected_columns = identifier_columns + df.columns[start_index:end_index + 1].tolist()
            column_prefix = '_selection'
        else:
            print("The first column must appear before the last column in the sheet.")
            exit()
    else:
        print("One or both of the specified columns are not found in the sheet.")
        exit()

# Create a new DataFrame with the selected columns
filtered_df = df[selected_columns]

# Rename the headers of non-identifier columns to a sequence of numbers starting from 1
non_identifier_columns = filtered_df.columns[len(identifier_columns):]  # Select non-identifier columns
new_column_names = identifier_columns + list(range(1, len(non_identifier_columns) + 1))
filtered_df.columns = new_column_names

# Construct the output file name
output_file = f'/content/proteinGroups_filtered{id_prefix}{column_prefix}.txt'

# Save the filtered DataFrame to a new TSV file with .txt extension
filtered_df.to_csv(output_file, sep='\t', index=False)

print(f"The new file has been saved as {output_file}")


#**Log transform** (optional)

This cell lets you apply multiple kinds of log transformations to the filtered abundances sheet

In [ ]:
import pandas as pd
import numpy as np
import re
import os
import math

# Define the input file path (TSV format)
input_file = '/content/proteinGroups_filtered_2id_selection.txt'

# Check if the input file exists
if not os.path.exists(input_file):
    print(f"Input file '{input_file}' does not exist.")
    exit()

# Load the TSV file into a pandas DataFrame
df = pd.read_csv(input_file, sep='\t')
print("TSV file loaded successfully.")

# Step 1: Ask the user for the number of identifier columns
num_identifiers = input("How many identifier columns are there? (Enter 1 or 2): ")

# Validate user input
if num_identifiers not in ['1', '2']:
    print("Invalid input. Please enter 1 or 2.")
    exit()

# Step 2: Ask the user to choose the log transformation type
print("Choose the type of log transformation:")
print("1. Natural (base e)")
print("2. Binary (base 2)")
print("3. Common (base 10)")
print("4. Custom base")
log_choice = input("Enter your choice (1, 2, 3, or 4): ")

# Validate user input
if log_choice not in ['1', '2', '3', '4']:
    print("Invalid input. Please enter 1, 2, 3, or 4.")
    exit()

# Define the log transformation function based on user choice
def log_transform(x, base):
    return np.log(x) / np.log(base)

# Step 3: Ask the user how to handle 0 or NaN values
print("How do you want to treat 0 or NaN values?")
print("1. Change to 1")
print("2. Conventional method/Treat as 1")
print("3. Type in a constant")
print("4. Leave as is")
zero_choice = input("Enter your choice (1, 2, 3, or 4): ")

# Validate user input
if zero_choice not in ['1', '2', '3', '4']:
    print("Invalid input. Please enter 1, 2, 3, or 4.")
    exit()

# Get the log base
if log_choice == '1':
    base = math.e
    log_suffix = '_Natural'
elif log_choice == '2':
    base = 2
    log_suffix = '_Binary'
elif log_choice == '3':
    base = 10
    log_suffix = '_Common'
else:
    base = float(input("Enter the custom base for the log transformation: "))
    log_suffix = f'_Base{int(base)}'

# Handle zero and NaN values
if zero_choice == '1':
    fill_value = 1
    fill_suffix = '_constant1'
elif zero_choice == '2':
    fill_value = 1
    fill_suffix = '_constant1'
elif zero_choice == '3':
    fill_value = float(input("Enter the constant value to replace 0 and NaN: "))
    fill_suffix = f'_constant{int(fill_value)}'
else:
    fill_value = None
    fill_suffix = ''

# Apply the chosen fill method
if fill_value is not None:
    df = df.fillna(fill_value)
    df.iloc[:, int(num_identifiers):] = df.iloc[:, int(num_identifiers):].replace(0, fill_value)

# Apply the log transformation to the appropriate columns
data_columns = df.columns[int(num_identifiers):]
df[data_columns] = df[data_columns].apply(lambda x: log_transform(x, base))

# Construct the output file name (TSV format with .txt extension)
output_file = f'/content/proteinGroups_filtered_2id_selection{log_suffix}{fill_suffix}.txt'

# Save the transformed DataFrame to a new TSV file
df.to_csv(output_file, sep='\t', index=False)

print(f"The transformed file has been saved as {output_file}")


#**Profile Plots**

This cell lets you plot profiles of various proteins from the filtered sheet. It also lets you pick alternate legend labels.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import os
import re

# Load the test_pfd.txt file
test_pfd_path = '/content/proteinGroups_filtered_2id_selection.txt'
test_pfd_df = pd.read_csv(test_pfd_path, sep='\t')

# Ask the user how many identifier columns are present
num_identifier_columns = int(input("How many identifier columns do you have (1 or 2)? "))

if num_identifier_columns == 2:
    # Display options for identifier column (for input)
    print("Select the identifier column you want to use to input protein names:")
    print(f"1: {test_pfd_df.columns[0]} (first column identifier)")
    print(f"2: {test_pfd_df.columns[1]} (second column identifier)")
    identifier_choice_input = input("Enter 1 or 2: ")

    # Display options for legend labels
    print("Select the identifier column(s) you want to use for legend labels:")
    print(f"1: {test_pfd_df.columns[0]} (first column identifier)")
    print(f"2: {test_pfd_df.columns[1]} (second column identifier)")
    print("3: Combine both columns as identifiers")
    identifier_choice_legend = input("Enter 1, 2, or 3: ")
else:
    identifier_choice_input = '1'
    identifier_choice_legend = '1'

# Prompt the user for the list of protein names
protein_names = input("Enter the protein names separated by commas: ").split(',')

# Clean up the protein names list
protein_names = [protein.strip() for protein in protein_names if protein]  # Remove any empty strings

# Choose the appropriate column for input identifiers
if identifier_choice_input == '1':
    input_column = test_pfd_df.columns[0]
elif identifier_choice_input == '2':
    input_column = test_pfd_df.columns[1]

# Find the matching rows in test_pfd.txt based on the input protein names
# Modified to check if IDs may be inside grouping (e.g., separated by semicolons)
matching_rows = test_pfd_df[test_pfd_df[input_column].apply(
    lambda cell: isinstance(cell, str) and any(protein in cell.split(';') for protein in protein_names)
)]

if matching_rows.empty:
    print("No matching proteins found in the selected identifier column.")
else:
    # Prepare the data for plotting
    x_values = test_pfd_df.columns[2:]  # X-axis values from the headers (excluding the first two columns)

    # Create the plot
    fig = go.Figure()

    for _, row in matching_rows.iterrows():
        y_values = row[2:].values  # Y-values for the heatmap
        y_values = pd.to_numeric(y_values, errors='coerce')  # Ensure values are numeric
        y_values = np.nan_to_num(y_values)  # Replace NaNs with 0

        # Determine the line label based on the user's choice
        if identifier_choice_legend == '1':
            line_label = row[test_pfd_df.columns[0]]  # First column identifier
        elif identifier_choice_legend == '2':
            line_label = row[test_pfd_df.columns[1]]  # Second column identifier
        elif identifier_choice_legend == '3':
            line_label = f"{row[test_pfd_df.columns[0]]} / {row[test_pfd_df.columns[1]]}"  # Combined identifiers

        # Add the line to the plot
        fig.add_trace(go.Scatter(x=x_values, y=y_values, mode='lines', name=line_label))

    # Determine if the file is log-transformed based on its name
    filename = os.path.basename(test_pfd_path)
    log_transformed_suffix = ""
    if "_Natural" in filename:
        log_transformed_suffix = "Natural Log Transformed"
    elif "_Binary" in filename:
        log_transformed_suffix = "Binary Log Transformed"
    elif "_Common" in filename:
        log_transformed_suffix = "Common Log Transformed"
    elif "_Base" in filename:
        # Extract the base number using a regular expression
        base_match = re.search(r"_Base(\d+)", filename)
        if base_match:
            base_number = base_match.group(1)  # Get the number after "_Base"
            log_transformed_suffix = f"Base{base_number} Log Transformed"

    # Set plot title and axis labels
    fig.update_layout(
        title=f'iBAQ Profile plot {protein_names} {log_transformed_suffix}',
        xaxis_title='Sample',
        yaxis_title='iBAQ Value'
    )

    # Show the plot
    fig.show()

If you have an excel workbook where each complex has a seprate sheet and each protein in the complex is in a different row (each protein may have different identifiers,) then this cell lets you plot profiles of various proteins from the *workbook*. It also lets you pick alternate legend labels.

In [ ]:
# By user complexes

import pandas as pd
import plotly.graph_objects as go
import numpy as np
import os
import re

# Load the complexes workbook
complexes_path = '/content/FGU13.xlsx'
complexes_xls = pd.ExcelFile(complexes_path)

# List available sheet names (complexes)
print("Available complexes:")
for i, sheet_name in enumerate(complexes_xls.sheet_names):
    print(f"{i + 1}: {sheet_name}")

# Prompt the user to select one or more complexes (sheets) by entering numbers
complex_choices = input("Enter the numbers of the complexes you are interested in (comma-separated, e.g., 1,3,5): ")
complex_choices = [int(num) - 1 for num in complex_choices.split(',')]  # Convert input to a list of indices

# Initialize lists to collect protein names and corresponding sheet names
all_protein_names = []

for complex_choice in complex_choices:
    complex_sheet_name = complexes_xls.sheet_names[complex_choice]
    complex_df = pd.read_excel(complexes_xls, sheet_name=complex_sheet_name)

    # Display available columns in the selected complex
    print(f"\nColumns in {complex_sheet_name}:")
    for i, col_name in enumerate(complex_df.columns):
        print(f"{i + 1}: {col_name}")

    # Prompt the user to select a column from each selected complex
    column_choice = int(input(f"Enter the number of the column you want to use for {complex_sheet_name}: ")) - 1
    selected_column_name = complex_df.columns[column_choice]

    # Get the names of the proteins from the selected column
    protein_names = complex_df[selected_column_name].dropna().tolist()
    all_protein_names.extend(protein_names)  # Append protein names for this sheet

# Load the abundances file
test_pfd_path = '/content/proteinGroups_filtered_2id_selection.txt'
test_pfd_df = pd.read_csv(test_pfd_path, sep='\t')

# Ask the user how many identifier columns are present
num_identifier_columns = int(input("\nHow many identifier columns do you have (1 or 2)? "))

if num_identifier_columns == 2:
    # Display options for the identifier columns in test_pfd.txt
    print("\nSelect the identifier column in test_pfd.txt to search for these names:")
    print(f"1: {test_pfd_df.columns[0]} (first column identifier)")
    print(f"2: {test_pfd_df.columns[1]} (second column identifier)")

    # Prompt the user to select the identifier column
    identifier_choice = int(input("Enter 1 or 2: ")) - 1
    identifier_column_name = test_pfd_df.columns[identifier_choice]

    # Ask user how they want to label the lines in the legend
    print("\nHow would you like the lines to be labeled?")
    print(f"1: Use {test_pfd_df.columns[0]}")
    print(f"2: Use {test_pfd_df.columns[1]}")
    print(f"3: Combine both {test_pfd_df.columns[0]} and {test_pfd_df.columns[1]}")

    label_choice = int(input("Enter 1, 2, or 3: "))
else:
    # Default to using the first column if only one identifier column is selected
    identifier_column_name = test_pfd_df.columns[0]
    label_choice = 1

# Find matching rows in test_pfd.txt based on the selected proteins
# Modified to check if IDs may be inside groupings (e.g., separated by semicolons)
matching_rows = test_pfd_df[test_pfd_df[identifier_column_name].apply(
    lambda cell: isinstance(cell, str) and any(protein in cell.split(';') for protein in all_protein_names)
)]

if matching_rows.empty:
    print("No matching proteins found in the selected identifier column.")
else:
    # Prepare the data for the profile plot
    x_values = test_pfd_df.columns[2:]  # X-axis values from the headers (excluding the first two columns)

    # Collect data for each matching row
    line_data = []
    line_labels = []

    for _, row in matching_rows.iterrows():
        y_values = row[2:].values  # Y-values for the line
        y_values = pd.to_numeric(y_values, errors='coerce')  # Ensure values are numeric
        y_values = np.nan_to_num(y_values)  # Replace NaNs with 0

        # Determine the row label based on the chosen labeling option
        if label_choice == 1:
            row_label = row[test_pfd_df.columns[0]]
        elif label_choice == 2:
            row_label = row[test_pfd_df.columns[1]]
        elif label_choice == 3:
            row_label = f"{row[test_pfd_df.columns[0]]} / {row[test_pfd_df.columns[1]]}"
        else:
            row_label = row[identifier_column_name]  # Fallback to the chosen identifier column

        # Append the data and label to the lists
        line_data.append(y_values)
        line_labels.append(row_label)

    # Determine if the file is log-transformed based on its name
    filename = os.path.basename(test_pfd_path)
    log_transformed_suffix = ""
    if "_Natural" in filename:
        log_transformed_suffix = "Natural Log Transformed"
    elif "_Binary" in filename:
        log_transformed_suffix = "Binary Log Transformed"
    elif "_Common" in filename:
        log_transformed_suffix = "Common Log Transformed"
    elif "_Base" in filename:
        # Extract the base number using a regular expression
        base_match = re.search(r"_Base(\d+)", filename)
        if base_match:
            base_number = base_match.group(1)  # Get the number after "_Base"
            log_transformed_suffix = f"Base{base_number} Log Transformed"

    # Create the profile plot
    fig = go.Figure()

    for i, y_values in enumerate(line_data):
        fig.add_trace(go.Scatter(
            x=x_values,
            y=y_values,
            mode='lines',
            name=line_labels[i]
        ))

    # Set plot title and axis labels
    fig.update_layout(
        title=f'Protein iBAQ Profile Plot for Selected Complexes {log_transformed_suffix}',
        xaxis_title='Sample',
        height=1000,
        width=4000,
        yaxis_title='iBAQ Value',
        yaxis=dict(automargin=True)  # Automatically adjust margins to avoid overlap
    )

    # Show the plot
    fig.show()


#**Heatmaps**

This cell lets you plot heatmaps of various proteins from the filtered sheet. It also lets you pick alternate legend labels.

In [ ]:
# By ID

import pandas as pd
import plotly.graph_objects as go
import numpy as np
import os
import re  # Import the regular expression module


# Function to limit the number of characters per label

def truncate_labels(labels, max_length=10):
    return [label if len(label) <= max_length else label[:max_length] + '...' for label in labels]

# Load the test_pfd.txt file
test_pfd_path = '/content/proteinGroups_filtered_2id_selection.txt'
test_pfd_df = pd.read_csv(test_pfd_path, sep='\t')

# Ask how many identifier columns are in the file
num_identifier_columns = input("How many columns do you have as identifiers? (1 or 2): ")

# Display options for identifier column (for input)
print("Select the identifier column you want to use to input protein names:")
if num_identifier_columns == '1':
    print(f"1: {test_pfd_df.columns[0]} (first column identifier)")
    identifier_choice_input = '1'  # Automatically set to the first column
elif num_identifier_columns == '2':
    print(f"1: {test_pfd_df.columns[0]} (first column identifier)")
    print(f"2: {test_pfd_df.columns[1]} (second column identifier)")
    identifier_choice_input = input("Enter 1 or 2: ")
else:
    raise ValueError("Invalid number of identifier columns. Please enter 1 or 2.")

# Display options for legend labels
print("Select the identifier column(s) you want to use for row labels:")
if num_identifier_columns == '1':
    print(f"1: {test_pfd_df.columns[0]} (first column identifier)")
    identifier_choice_legend = '1'  # Automatically set to the first column
elif num_identifier_columns == '2':
    print(f"1: {test_pfd_df.columns[0]} (first column identifier)")
    print(f"2: {test_pfd_df.columns[1]} (second column identifier)")
    print("3: Combine both columns as identifiers")
    identifier_choice_legend = input("Enter 1, 2, or 3: ")
else:
    raise ValueError("Invalid number of identifier columns. Please enter 1 or 2.")

# Prompt the user for the list of protein names
protein_names = input("Enter the protein names separated by commas: ").split(',')

# Clean up the protein names list and reverse the order
protein_names = [protein.strip() for protein in protein_names if protein]  # Remove any empty strings
protein_names = protein_names[::-1]  # Reverse the input order of proteins

# Choose the appropriate column for input identifiers
if identifier_choice_input == '1':
    input_column = test_pfd_df.columns[0]
elif identifier_choice_input == '2':
    input_column = test_pfd_df.columns[1]

# Find the matching rows in test_pfd.txt based on the input protein names
# Modified to check if IDs may be inside grouping (e.g., separated by semicolons)
matching_rows = test_pfd_df[test_pfd_df[input_column].apply(
    lambda cell: isinstance(cell, str) and any(protein in cell.split(';') for protein in protein_names)
)]

if matching_rows.empty:
    print("No matching proteins found in the selected identifier column.")
else:
    # Prepare the data for the heatmap
    x_values = test_pfd_df.columns[2:]  # X-axis values from the headers (excluding the first two columns)
    # Collect data for each row in the heatmap
    heatmap_data = []
    row_labels = []

    for _, row in matching_rows.iterrows():
        y_values = row[2:].values  # Y-values for the heatmap
        y_values = pd.to_numeric(y_values, errors='coerce')  # Ensure values are numeric
        y_values = np.nan_to_num(y_values)  # Replace NaNs with 0

        # Determine the row label based on the user's choice
        if identifier_choice_legend == '1':
            row_label = row[test_pfd_df.columns[0]]  # First column identifier
        elif identifier_choice_legend == '2':
            row_label = row[test_pfd_df.columns[1]]  # Second column identifier
        elif identifier_choice_legend == '3':
            row_label = f"{row[test_pfd_df.columns[0]]} / {row[test_pfd_df.columns[1]]}"  # Combined identifiers

        # Append the data and label to the lists
        heatmap_data.append(y_values)
        row_labels.append(row_label)


    # Truncate the row labels if they exceed a certain length

    max_char_length = 25  # You can adjust the maximum number of characters allowed per label

    row_labels = truncate_labels(row_labels, max_length=max_char_length) # Hash this out if you dont want label truncation


    # Check if log transformation is to be applied and identify the base
    filename = os.path.basename(test_pfd_path)
    log_transformed_suffix = ""
    apply_log_transform = True  # Set this to True or False depending on your preference

    if apply_log_transform:
        # Determine the log transformation type based on file name
        if "_Natural" in filename:
            log_transformed_suffix = "Natural Log Transformed"
        elif "_Binary" in filename:
            log_transformed_suffix = "Binary Log Transformed"
        elif "_Common" in filename:
            log_transformed_suffix = "Common Log Transformed"
        elif "_Base" in filename:
            # Extract the base number using a regular expression
            base_match = re.search(r"_Base(\d+)", filename)
            if base_match:
                base_number = base_match.group(1)  # Get the number after "_Base"
                log_transformed_suffix = f"Base{base_number} Log Transformed"

        # Apply log transformation to heatmap data
        #heatmap_data = [np.log1p(row) for row in heatmap_data]  # Apply log1p to each row individually

        # Count the number of proteins to be visualized
        num_proteins = len(row_labels)

        # Define a scaling factor to adjust the plot height and a minimum height
        height_scaling_factor = 40  # You can change this number as needed
        min_plot_height = 200  # Minimum height for the plot

        # Calculate the height of the plot based on the number of proteins
        plot_height = max(num_proteins * height_scaling_factor, min_plot_height)


    # Create the heatmap
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data,
        x=x_values,
        y=row_labels,
        colorscale='Viridis',  # You can change the colorscale if desired
        hoverinfo="x+y+z"
    ))

    # Set plot title and axis labels with automargin enabled
    fig.update_layout(
        title=f'iBAQ Heatmap {protein_names} {log_transformed_suffix}',
        xaxis_title='Sample',
        yaxis_title='Proteins',
        #width=700,  # You can adjust this value
        #height=1000,  # You can adjust this value based on the number of proteins
        yaxis=dict(automargin=True)  # Automatically adjust margins to avoid overlap
    )

    # Show the plot
    fig.show()


If you have an excel workbook where each complex has a seprate sheet and each protein in the complex is in a different row (each protein may have different identifiers,) then this cell lets you plot profiles of various proteins from the *workbook*. It also lets you pick alternate legend labels.

In [ ]:
# By user complexes, single label

import pandas as pd
import plotly.graph_objects as go
import numpy as np
import os
import re  # Import the regular expression module

# Function to limit the number of characters in tick labels
def limit_label_length(labels, max_length):
    return [label if len(label) <= max_length else label[:max_length] + "..." for label in labels]

# Load the Excel workbook
complexes_path = '/content/FGU13.xlsx'
complexes_xls = pd.ExcelFile(complexes_path)

# List available sheet names (complexes)
print("Available complexes:")
for i, sheet_name in enumerate(complexes_xls.sheet_names):
    print(f"{i + 1}: {sheet_name}")

# Prompt the user to select one or more complexes (sheets) by entering numbers
complex_choices = input("Enter the numbers of the complexes you are interested in (comma-separated, e.g., 1,3,5): ")
complex_choices = [int(num) - 1 for num in complex_choices.split(',')]  # Convert input to a list of indices

# Load the selected sheets and keep track of the order of protein names
all_protein_names = []

for complex_choice in complex_choices:
    complex_sheet_name = complexes_xls.sheet_names[complex_choice]
    complex_df = pd.read_excel(complexes_xls, sheet_name=complex_sheet_name)

    # Display available columns in the selected complex
    print(f"\nColumns in {complex_sheet_name}:")
    for i, col_name in enumerate(complex_df.columns):
        print(f"{i + 1}: {col_name}")

    # Prompt the user to select a column from each selected complex
    column_choice = int(input(f"Enter the number of the column you want to use for {complex_sheet_name}: ")) - 1
    selected_column_name = complex_df.columns[column_choice]

    # Get the names of the proteins from the selected column and maintain order
    protein_names = complex_df[selected_column_name].dropna().tolist()
    all_protein_names.append(protein_names)  # Append the list of proteins for this sheet

# Flatten the list of protein names while maintaining order across sheets
ordered_protein_names = [protein for sublist in all_protein_names for protein in sublist]

# Invert the order of the protein names
ordered_protein_names.reverse()

# Load the test_pfd.txt file
test_pfd_path = '/content/newtest_pfd2_Binary.txt'
test_pfd_df = pd.read_csv(test_pfd_path, sep='\t')

# Ask how many identifier columns are in the file
num_identifier_columns = input("How many columns do you have as identifiers? (1 or 2): ")

# Display options for identifier column (for input)
print("\nSelect the identifier column in test_pfd.txt to search for these names:")
if num_identifier_columns == '1':
    print(f"1: {test_pfd_df.columns[0]} (first column identifier)")
    identifier_choice = '1'  # Automatically set to the first column
elif num_identifier_columns == '2':
    print(f"1: {test_pfd_df.columns[0]} (first column identifier)")
    print(f"2: {test_pfd_df.columns[1]} (second column identifier)")
    identifier_choice = input("Enter 1 or 2: ")
else:
    raise ValueError("Invalid number of identifier columns. Please enter 1 or 2.")

# Choose the appropriate column for input identifiers
identifier_column_name = test_pfd_df.columns[int(identifier_choice) - 1]

# Ask the user how they would like the lines to be labeled (three options)
print("\nHow would you like the lines to be labeled?")
print(f"1: {test_pfd_df.columns[0]} (first column identifier)")
if num_identifier_columns == '2':
    print(f"2: {test_pfd_df.columns[1]} (second column identifier)")
    print("3: Combine both columns as identifiers")
    label_choice = input("Enter 1, 2, or 3: ")
else:
    label_choice = '1'  # Automatically set to first column if only one column exists

# Find matching rows in test_pfd.txt based on the ordered protein names
matching_rows = []

for protein in ordered_protein_names:
    matches = test_pfd_df[test_pfd_df[identifier_column_name].apply(
        lambda cell: isinstance(cell, str) and protein in cell.split(';'))
    ]
    if not matches.empty:
        matching_rows.append(matches.iloc[0])  # Get the first match for each protein

if not matching_rows:
    print("No matching proteins found in the selected identifier column.")
else:
    # Prepare the data for the heatmap
    x_values = test_pfd_df.columns[2:]  # X-axis values from the headers (excluding the first two columns)

    # Collect data for the heatmap
    heatmap_data = []
    y_labels = []

    for row in matching_rows:
        y_values = row[2:].values  # Y-values for the heatmap
        y_values = pd.to_numeric(y_values, errors='coerce')  # Ensure values are numeric
        y_values = np.nan_to_num(y_values)  # Replace NaNs with 0

        # Determine the row label based on the user's choice
        if label_choice == '1':
            row_label = row[test_pfd_df.columns[0]]  # First column identifier
        elif label_choice == '2':
            row_label = row[test_pfd_df.columns[1]]  # Second column identifier
        elif label_choice == '3':
            row_label = f"{row[test_pfd_df.columns[0]]} / {row[test_pfd_df.columns[1]]}"  # Combined identifiers

        # Append the data and label to the lists
        heatmap_data.append(y_values)
        y_labels.append(row_label)

    # Limit the label length (e.g., to 20 characters)
    max_label_length = 20  # You can change this value
    y_labels = limit_label_length(y_labels, max_label_length)

    # Check if log transformation is to be applied and identify the base
    filename = os.path.basename(test_pfd_path)
    log_transformed_suffix = ""
    apply_log_transform = True  # Set this to True or False depending on your preference

    if apply_log_transform:
        # Determine the log transformation type based on file name
        if "_Natural" in filename:
            log_transformed_suffix = "Natural Log Transformed"
        elif "_Binary" in filename:
            log_transformed_suffix = "Binary Log Transformed"
        elif "_Common" in filename:
            log_transformed_suffix = "Common Log Transformed"
        elif "_Base" in filename:
            # Extract the base number using a regular expression
            base_match = re.search(r"_Base(\d+)", filename)
            if base_match:
                base_number = base_match.group(1)  # Get the number after "_Base"
                log_transformed_suffix = f"Base{base_number} Log Transformed"

        # Apply log transformation to heatmap data if required
        # heatmap_data = [np.log1p(row) for row in heatmap_data]  # Example: log transformation

        # Define a scaling factor to adjust the plot height and a minimum height
        height_scaling_factor = 40  # You can change this number as needed
        min_plot_height = 200  # Minimum height for the plot

        # Calculate the height of the plot based on the number of proteins
        plot_height = max(len(y_labels) * height_scaling_factor, min_plot_height)

    # Create the heatmap
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data,
        x=x_values,
        y=y_labels,
        colorscale='Viridis'
    ))

    # Set plot title and axis labels
    fig.update_layout(
        title=f'Protein iBAQ Heatmap for Selected Complexes - {log_transformed_suffix}',
        xaxis_title='Sample',
        yaxis_title='Proteins',
        width=2800,  # You can adjust this value
        height=1200,  # Adjust the plot height dynamically
        yaxis=dict(automargin=True)  # Automatically adjust margins to avoid overlap
    )

    # Show the plot
    fig.show()


This cell is similar to the previous one except it adds extended labels with complex names

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import os
import re  # Import the regular expression module

# Function to limit the number of characters in tick labels and append sheet name
def limit_label_length(labels, max_length, sheet_names):
    return [
        (label if len(label) <= max_length else label[:max_length] + "...") + f" ({sheet})"
        for label, sheet in zip(labels, sheet_names)
    ]

# Load the Excel workbook
complexes_path = '/content/FGU13.xlsx'
complexes_xls = pd.ExcelFile(complexes_path)

# List available sheet names (complexes)
print("Available complexes:")
for i, sheet_name in enumerate(complexes_xls.sheet_names):
    print(f"{i + 1}: {sheet_name}")

# Prompt the user to select one or more complexes (sheets) by entering numbers
complex_choices = input("Enter the numbers of the complexes you are interested in (comma-separated, e.g., 1,3,5): ")
complex_choices = [int(num) - 1 for num in complex_choices.split(',')]  # Convert input to a list of indices

# Load the selected sheets and keep track of the order of protein names
all_protein_names = []
all_sheet_names = []  # To store the corresponding sheet name for each protein

for complex_choice in complex_choices:
    complex_sheet_name = complexes_xls.sheet_names[complex_choice]
    complex_df = pd.read_excel(complexes_xls, sheet_name=complex_sheet_name)

    # Display available columns in the selected complex
    print(f"\nColumns in {complex_sheet_name}:")
    for i, col_name in enumerate(complex_df.columns):
        print(f"{i + 1}: {col_name}")

    # Prompt the user to select a column from each selected complex
    column_choice = int(input(f"Enter the number of the column you want to use for {complex_sheet_name}: ")) - 1
    selected_column_name = complex_df.columns[column_choice]

    # Get the names of the proteins from the selected column and maintain order
    protein_names = complex_df[selected_column_name].dropna().tolist()
    all_protein_names.append(protein_names)  # Append the list of proteins for this sheet
    all_sheet_names.extend([complex_sheet_name] * len(protein_names))  # Append sheet name for each protein

# Flatten the list of protein names while maintaining order across sheets
ordered_protein_names = [protein for sublist in all_protein_names for protein in sublist]

# Invert the order of the protein names
ordered_protein_names.reverse()
all_sheet_names.reverse()  # Reverse the corresponding sheet names

# Load the test_pfd.txt file
test_pfd_path = '/content/newtest_pfd2_Binary.txt'
test_pfd_df = pd.read_csv(test_pfd_path, sep='\t')

# Ask how many identifier columns are in the file
num_identifier_columns = input("How many columns do you have as identifiers? (1 or 2): ")

# Display options for identifier column (for input)
print("\nSelect the identifier column in test_pfd.txt to search for these names:")
if num_identifier_columns == '1':
    print(f"1: {test_pfd_df.columns[0]} (first column identifier)")
    identifier_choice = '1'  # Automatically set to the first column
elif num_identifier_columns == '2':
    print(f"1: {test_pfd_df.columns[0]} (first column identifier)")
    print(f"2: {test_pfd_df.columns[1]} (second column identifier)")
    identifier_choice = input("Enter 1 or 2: ")
else:
    raise ValueError("Invalid number of identifier columns. Please enter 1 or 2.")

# Choose the appropriate column for input identifiers
identifier_column_name = test_pfd_df.columns[int(identifier_choice) - 1]

# Ask the user how they would like the lines to be labeled (three options)
print("\nHow would you like the lines to be labeled?")
print(f"1: {test_pfd_df.columns[0]} (first column identifier)")
if num_identifier_columns == '2':
    print(f"2: {test_pfd_df.columns[1]} (second column identifier)")
    print("3: Combine both columns as identifiers")
    label_choice = input("Enter 1, 2, or 3: ")
else:
    label_choice = '1'  # Automatically set to first column if only one column exists

# Find matching rows in test_pfd.txt based on the ordered protein names
matching_rows = []

for protein in ordered_protein_names:
    matches = test_pfd_df[test_pfd_df[identifier_column_name].apply(
        lambda cell: isinstance(cell, str) and protein in cell.split(';'))
    ]
    if not matches.empty:
        matching_rows.append(matches.iloc[0])  # Get the first match for each protein

if not matching_rows:
    print("No matching proteins found in the selected identifier column.")
else:
    # Prepare the data for the heatmap
    x_values = test_pfd_df.columns[2:]  # X-axis values from the headers (excluding the first two columns)

    # Collect data for the heatmap
    heatmap_data = []
    y_labels = []

    for row in matching_rows:
        y_values = row[2:].values  # Y-values for the heatmap
        y_values = pd.to_numeric(y_values, errors='coerce')  # Ensure values are numeric
        y_values = np.nan_to_num(y_values)  # Replace NaNs with 0

        # Determine the row label based on the user's choice
        if label_choice == '1':
            row_label = row[test_pfd_df.columns[0]]  # First column identifier
        elif label_choice == '2':
            row_label = row[test_pfd_df.columns[1]]  # Second column identifier
        elif label_choice == '3':
            row_label = f"{row[test_pfd_df.columns[0]]} / {row[test_pfd_df.columns[1]]}"  # Combined identifiers

        # Append the data and label to the lists
        heatmap_data.append(y_values)
        y_labels.append(row_label)

    # Limit the label length (e.g., to 20 characters) and append the sheet name
    max_label_length = 20  # You can change this value
    y_labels = limit_label_length(y_labels, max_label_length, all_sheet_names)

    # Check if log transformation is to be applied and identify the base
    filename = os.path.basename(test_pfd_path)
    log_transformed_suffix = ""
    apply_log_transform = True  # Set this to True or False depending on your preference

    if apply_log_transform:
        # Determine the log transformation type based on file name
        if "_Natural" in filename:
            log_transformed_suffix = "Natural Log Transformed"
        elif "_Binary" in filename:
            log_transformed_suffix = "Binary Log Transformed"
        elif "_Common" in filename:
            log_transformed_suffix = "Common Log Transformed"
        elif "_Base" in filename:
            # Extract the base number using a regular expression
            base_match = re.search(r"_Base(\d+)", filename)
            if base_match:
                base_number = base_match.group(1)  # Get the number after "_Base"
                log_transformed_suffix = f"Base{base_number} Log Transformed"

        # Define a scaling factor to adjust the plot height and a minimum height
        height_scaling_factor = 40  # You can change this number as needed
        min_plot_height = 200  # Minimum height for the plot

        # Calculate the height of the plot based on the number of proteins
        plot_height = max(len(y_labels) * height_scaling_factor, min_plot_height)

    # Create the heatmap
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data,
        x=x_values,
        y=y_labels,
        colorscale='Viridis'
    ))

    # Set plot title and axis labels
    fig.update_layout(
        title=f'Protein iBAQ Heatmap for Selected Complexes - {log_transformed_suffix}',
        xaxis_title='Sample',
        yaxis_title='Proteins',
        width=2800,  # You can adjust this value
        height=1200,  # Adjust the plot height dynamically
        yaxis=dict(automargin=True)  # Automatically adjust margins to avoid overlap
    )

    # Show the plot
    fig.show()
